In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt


In [4]:
# Use error_bad_lines=False arg to omit params for runs where the params got concatenated with run data
data = pd.read_csv("../data/survivability/adult/data-habitat.2020.May.28.04_23_23")
data.head()

,run,randomSeed,tick_lifestage,large_host_density,tick_count,habitat_suitability
0,1,1.500214e+09,adult,1.0,2.0,-1.00
1,9,1.502528e+09,adult,1.0,2.0,0.08
2,17,1.504877e+09,adult,1.0,3.0,0.05
3,25,1.507203e+09,adult,1.0,4.0,0.02
4,33,1.509553e+09,adult,1.0,4.0,0.10


In [ ]:
# Loop through groupby and print different parts
# for name, group in grouped_data:
#     print("Run:", name[:1], "Tick:", name[1:])
#     print("Group:\n",group)
#     valcounts= group['Lifestate'].value_counts().to_dict()
#     print(valcounts)
#     lk_vals = [[k,v] for k, v in valcounts.items()]

In [5]:
by_run = data.groupby('run')
runs = data['run'].unique()
runs

array([ 1,  9, 17, 25])

In [8]:
#by_run['tick'].count()

for group in data.groupby('run'):
    df = group[1]
    print("Run {} made it {} 'days'".format(group[0], df['tick'].max()))


Run 1 made it 86.0 'days'
Run 9 made it 86.0 'days'
Run 17 made it 450.0 'days'
Run 25 made it 366.0 'days'


In [9]:
# get a single group to work with
group1 = by_run.get_group(1)
counts = group1['Lifestate'].value_counts().to_frame() # can also add .to_dict()
counts
#----------------------------------------------------------------------------------


,Lifestate
adult,90


In [7]:
# drop the columns we don't care about for this particular plot
group1 = group1[group1.Lifestate != "egg"]
g1_df = group1.drop(["Lat", "Long", "habitat_sample", "run"], axis=1)
# g1_df = group1.drop([ "habitat_sample", "run"], axis=1)
g1_df.head()

,tick,Lifestate
4634,1.0,nymph
4635,1.0,nymph
4636,1.0,nymph
4637,1.0,nymph
4638,1.0,nymph


In [ ]:
# create a column for days
g1_df['day'] = g1_df.apply(lambda x: (x['tick'] + (x['tick'] // 90) * 275), axis=1)

# g1_df['day'] = g1_df.apply(lambda x: x['tick'] + 275 if x['tick'] > 90 else x['tick'], axis=1)  # NO, only good for 2 years

In [ ]:
# Create the plot
# https://stackoverflow.com/questions/5656798/python-matplotlib-is-there-a-way-to-make-a-discontinuous-axis
# https://matplotlib.org/devdocs/gallery/subplots_axes_and_figures/broken_axis.html

matplotlib.use('Qt5Agg')

d = g1_df.drop('tick', axis=1)
days_df = pd.get_dummies(d, columns=['Lifestate'])
by_day = days_df.groupby('day')

fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(12,7), sharey=True)
by_day.sum().plot(logy=True, ax=ax1)
by_day.sum().plot(logy=True, ax=ax2, legend=False)
by_day.sum().plot(logy=True, ax=ax3, legend=False)
# by_day.sum().plot(logy=True, ax=ax4, legend=False)
min = days_df['day'].max() / 90
# ax1.set_xlim(0,90)
# ax2.set_xlim(365,)
ax1.set_xlim(0,90)
ax2.set_xlim(367,456)
ax3.set_xlim(732,)
# ax4.set_xlim(1095,)
# ax5.set_xlim(1185,1275)

# Now we want to hide spines and ticks
ax1.spines['right'].set_visible(False)
ax2.spines['left'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax3.spines['left'].set_visible(False)
# ax3.spines['right'].set_visible(False)
# ax4.spines['left'].set_visible(False)

ax1.x_label.set_visible(False)
ax3.x_label.set_visible(False)

ax1.yaxis.tick_left()
ax3.yaxis.tick_right()  # TODO hide the ticks on ax2

# make the gap between axes smaller
plt.subplots_adjust(wspace=0.05)

# add cutout lines
d = 0.015
kwargs = dict(transform=ax1.transAxes, color='k', clip_on=False)
ax1.plot((1-d, 1+d), (-d, +d), **kwargs)
ax1.plot((1-d, 1+d), (1-d, 1+d), **kwargs)

kwargs.update(transform=ax2.transAxes)
ax2.plot((-d,d),(-d,+d), **kwargs) # top-right diagonal
ax2.plot((-d,d),(1-d,1+d), **kwargs) # bottom-right diagonal

plt.show(block=True)  # Another way to get popup to stay (https://stackoverflow.com/questions/33062762/plot-in-pandas-immediately-closes/33062819)


In [ ]:
# Handle categoricals by getting dummy variables for them
# matplotlib.use('Qt5Agg')  # Force default backend for matplotlib to get plot as a popup
d = pd.get_dummies(g1_df, columns=['Lifestate']) # create a new dataframe with dummy variables
# d.head()
gr = d.groupby('tick')
plot = gr.sum().plot(logy=True, title="Ixode Survivability")
plot.set_xlabel("days")
plot.set_ylabel("Lifestate count")


In [ ]:
params = " Lifestage: {}\n Host Density: {}\n Ixode Count: {}\n Habitat Suitability: {}".format(
                  "nymph", 0.5, 4, 0.05)
fig,ax = plt.subplots(figsize=(12,7)) # ax is the plot, plt is the figure the plot goes in
gr.sum().plot(logy=True, title="AAAAAAAAAAASurvivability", ax=ax)
ax.set_title("Survivability", fontsize=16, fontweight='bold')
ax.set_xlabel("Days", fontsize=12)
ax.set_ylabel("Lifestate count", fontsize=12)
# props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
props = dict( facecolor='wheat', alpha=0.5)
ax.text(17,1100, params, fontsize=10, bbox=props)
postfix = 00
fig.savefig('/home/hillari/PycharmProjects/tick-ABMs/data/plots/testplot' + str(postfix) + '.png')


In [ ]:
# calculate something like this
# tick_nums = [1, 70, 145, 185, 215, 375, 450]
# day_nums = [x + ( (x // 90) * 275 ) for x in tick_nums]
# print(day_nums)

In [8]:
# calculate something like this
# tick_nums = [1, 70, 145, 185, 215, 375, 450]
# day_nums = [x + ( (x // 90) * 275 ) for x in tick_nums]
# print(day_nums)

In [1]:
# Create the plot
# https://stackoverflow.com/questions/5656798/python-matplotlib-is-there-a-way-to-make-a-discontinuous-axis
# https://matplotlib.org/devdocs/gallery/subplots_axes_and_figures/broken_axis.html

matplotlib.use('Qt5Agg')

d = g1_df.drop('tick', axis=1)
days_df = pd.get_dummies(d, columns=['Lifestate'])
by_day = days_df.groupby('day')

fig, (ax1, ax2, ax3) = plt.subplots(1,3,figsize=(12,7), sharey=True)
by_day.sum().plot(logy=True, ax=ax1)
by_day.sum().plot(logy=True, ax=ax2, legend=False)
by_day.sum().plot(logy=True, ax=ax3, legend=False)
# by_day.sum().plot(logy=True, ax=ax4, legend=False)
min = days_df['day'].max() / 90
# ax1.set_xlim(0,90)
# ax2.set_xlim(365,)
ax1.set_xlim(0,90)
ax2.set_xlim(367,456)
ax3.set_xlim(732,)
# ax4.set_xlim(1095,)
# ax5.set_xlim(1185,1275)

# Now we want to hide spines and ticks
ax1.spines['right'].set_visible(False)
ax2.spines['left'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax3.spines['left'].set_visible(False)
# ax3.spines['right'].set_visible(False)
# ax4.spines['left'].set_visible(False)

ax1.x_label.set_visible(False)
ax3.x_label.set_visible(False)

ax1.yaxis.tick_left()
ax3.yaxis.tick_right()  # TODO hide the ticks on ax2

# make the gap between axes smaller
plt.subplots_adjust(wspace=0.05)

# add cutout lines
d = 0.015
kwargs = dict(transform=ax1.transAxes, color='k', clip_on=False)
ax1.plot((1-d, 1+d), (-d, +d), **kwargs)
ax1.plot((1-d, 1+d), (1-d, 1+d), **kwargs)

kwargs.update(transform=ax2.transAxes)
ax2.plot((-d,d),(-d,+d), **kwargs) # top-right diagonal
ax2.plot((-d,d),(1-d,1+d), **kwargs) # bottom-right diagonal

plt.show(block=True)  # Another way to get popup to stay (https://stackoverflow.com/questions/33062762/plot-in-pandas-immediately-closes/33062819)


NameError: name 'matplotlib' is not defined

In [ ]:
# Handle categoricals by getting dummy variables for them
# matplotlib.use('Qt5Agg')  # Force default backend for matplotlib to get plot as a popup
d = pd.get_dummies(g1_df, columns=['Lifestate']) # create a new dataframe with dummy variables
# d.head()
gr = d.groupby('tick')
plot = gr.sum().plot(logy=True, title="Ixode Survivability")
plot.set_xlabel("days")
plot.set_ylabel("Lifestate count")


In [ ]:
params = " Lifestage: {}\n Host Density: {}\n Ixode Count: {}\n Habitat Suitability: {}".format(
                  "nymph", 0.5, 4, 0.05)
fig,ax = plt.subplots(figsize=(12,7)) # ax is the plot, plt is the figure the plot goes in
gr.sum().plot(logy=True, title="AAAAAAAAAAASurvivability", ax=ax)
ax.set_title("Survivability", fontsize=16, fontweight='bold')
ax.set_xlabel("Days", fontsize=12)
ax.set_ylabel("Lifestate count", fontsize=12)
# props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
props = dict( facecolor='wheat', alpha=0.5)
ax.text(17,1100, params, fontsize=10, bbox=props)
postfix = 00
fig.savefig('/home/hillari/PycharmProjects/tick-ABMs/data/plots/testplot' + str(postfix) + '.png')


In [ ]:
# calculate something like this
# tick_nums = [1, 70, 145, 185, 215, 375, 450]
# day_nums = [x + ( (x // 90) * 275 ) for x in tick_nums]
# print(day_nums)